# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:

city_data_path = "city_data.csv"
city_df = pd.read_csv(city_data_path)
city_df.head()

,Unnamed: 0,City Name,Lat,Lon,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Busselton,-33.6500,115.3333,66.99,73,45,3.00,AU,1614298115
1,1,Bubaque,11.2833,-15.8333,76.62,69,0,14.88,GW,1614298148
2,2,Jamestown,42.0970,-79.2353,26.60,63,1,4.61,US,1614298127
3,3,Atuona,-9.8000,-139.0333,80.33,79,3,12.24,PF,1614298151
4,4,Hobart,-42.8794,147.3294,63.00,82,75,14.97,AU,1614298092


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [5]:
gmaps.configure(api_key=g_key)

In [20]:
coordinates = city_df[["Lat", "Lon"]].astype(float)
humidity =city_df['Humidity'].astype(float)

figure = gmaps.figure()

heat_map = gmaps.heatmap_layer(coordinates, weights=humidity,
                              dissipating=False, max_intensity=100,
                              point_radius=3)
figure.add_layer(heat_map)

figure

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [25]:
ideal_vacation_spot_df = city_df.loc[city_df['Max Temperature'] > 70]
ideal_vacation_spot_df = ideal_vacation_spot_df.loc[ideal_vacation_spot_df['Max Temperature'] < 80]
ideal_vacation_spot_df = ideal_vacation_spot_df.loc[ideal_vacation_spot_df['Wind Speed'] < 10]
ideal_vacation_spot_df = ideal_vacation_spot_df.loc[ideal_vacation_spot_df['Cloudiness'] == 0]
ideal_vacation_spot_df = ideal_vacation_spot_df.loc[ideal_vacation_spot_df['Humidity'] < 75]

ideal_vacation_spot_df

,Unnamed: 0,City Name,Lat,Lon,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Date
32,32,Sompeta,18.9333,84.6000,73.53,48,0,7.40,IN,1614298187
47,47,Kolondiéba,11.0882,-6.8926,75.31,17,0,7.00,ML,1614298207
65,65,Coyhaique,-45.5752,-72.0662,71.60,46,0,4.61,CL,1614298014
106,106,San Jorge,-31.8962,-61.8598,78.01,58,0,8.81,AR,1614298292
135,135,Paraopeba,-19.2757,-44.4033,77.00,73,0,9.22,BR,1614298337
173,173,São Filipe,14.8961,-24.4956,71.62,72,0,7.29,CV,1614298390
281,281,Rio Grande,-32.0350,-52.0986,70.75,72,0,8.23,BR,1614298506
314,314,Iquique,-20.2208,-70.1431,71.60,60,0,9.22,CL,1614298592
386,386,Chabahar,25.2919,60.6430,73.89,70,0,6.17,IR,1614298690


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
